# === USER CONFIGURATION ===


In [ ]:
# # === USER CONFIGURATION ===
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, please read the README_Notebooks.md file before
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

WORKDIR = "PATH/TO/WORKDIR"  # Working directory for sen2vm processing
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, the structure of the WORKDIR shall respect the one described in README_Notebooks.md
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

# Path to downloaded product
PATH_L1B_DATA = "PATH/TO/L1B/PRODUCT"
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, for now PATH_L1B_DATA shall be inside WORKID/DATA and respect the one described in README_Notebooks.md
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

# Output folder chosen by the user
INVERSE_OUTPUT_FOLDER = "PATH/TO/OUTPUT/FOLDER"  # Used only if GRID_MODE = "inverse"


# === SEN2VM OPTIONS ===

GRID_MODE = "direct"  # direct or inverse
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, for now only direct shall be used with a full product (no missing granules)
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

UTM_EPSG = 32626 # UTM zone EPSG code for the ROI
LOCATION = {
    "ul_x": 366305,  
    "ul_y": 4272412,  
    "lr_x": 414745,   
    "lr_y": 4244057   
}

STEPS = {
    "10m_bands": 10,
    "20m_bands": 20,
    "60m_bands": 60
}


# === GDAL ORTHO OPTIONS ===

ORTHO_SETTINGS = {
    "keep_bands": ["B01" , "B02", "B03", "B04"], # list of bands to keep for orthorectification
}
 


# GIPP database download (Optionnal)

In [2]:
# === GIPPs ===
# This step is optionnal if the Database was already downloaded or if you want to use your own GIPP
# Please note that this current notebook will search for a subfolder with mission S2[A/B/C] inside the GIPP folder

import os
import shutil
import tarfile
import re
from datetime import datetime


gipp_dir = os.path.join(WORKDIR, "DATA")     
os.makedirs(gipp_dir, exist_ok=True)             

# =====================================================================
# 1) CLONE sen2vm-gipp-database
# =====================================================================

gipp_repo_name = "sen2vm-gipp-database"
gipp_repo_path = os.path.join(gipp_dir, gipp_repo_name)

# Delete repository if exists
if os.path.exists(gipp_repo_path):
    print(f"Removing existing repository: {gipp_repo_path}")
    shutil.rmtree(gipp_repo_path)

# Clone repository fresh
print("Cloning sen2vm-gipp-database...")
!git clone https://github.com/sen2vm/sen2vm-gipp-database.git {gipp_repo_path}
print("Clone complete.\n")



print("GIPP processing finished successfully.")


Removing existing repository: /home/avinas/Documents/sen2vm-fix/DATA/sen2vm-gipp-database
Cloning sen2vm-gipp-database...
Cloning into '/home/avinas/Documents/sen2vm-fix/DATA/sen2vm-gipp-database'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 85 (delta 5), reused 79 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 17.51 MiB | 7.09 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Clone complete.

GIPP processing finished successfully.


# IERS Download (optionnal)

In [3]:
# === DOWNLOAD IERS ===

import os
import re
import requests
from datetime import datetime

DATA_DIR = os.path.join(WORKDIR, "DATA")

if not os.path.exists(DATA_DIR):
    raise RuntimeError(f"DATA directory not found: {DATA_DIR}")

print("Bulletin output directory:", DATA_DIR)

# =====================================================================
# 0. REMOVE EXISTING IERS BULLETINS
# =====================================================================

for f in os.listdir(DATA_DIR):
    if f.startswith("bulletina-") and f.endswith(".txt"):
        os.remove(os.path.join(DATA_DIR, f))
        print("Removed old bulletin A:", f)

    if f.startswith("bulletinb-") and f.endswith(".txt"):
        os.remove(os.path.join(DATA_DIR, f))
        print("Removed old bulletin B:", f)

print("Cleanup of old bulletins complete.\n")

# =====================================================================
# 2. EXTRACT PRODUCT DATE (FROM DATASTRIP)
# =====================================================================

datastrip_dir = os.path.join(PATH_L1B_DATA, "DATASTRIP")

if not os.path.isdir(datastrip_dir):
    raise RuntimeError(f"DATASTRIP directory not found: {datastrip_dir}")

datastrip_entries = os.listdir(datastrip_dir)
if not datastrip_entries:
    raise RuntimeError(f"No DATASTRIP found in: {datastrip_dir}")

# Take the first DATASTRIP product
datastrip_name = datastrip_entries[0]

match = re.search(r"_S(\d{8})T\d{6}_", datastrip_name)
if not match:
    raise RuntimeError("Could not extract product date from DATASTRIP name.")

product_date_str = match.group(1)

year = int(product_date_str[:4])
month = int(product_date_str[4:6])
day = int(product_date_str[6:8])

product_date = datetime(year, month, day)

print("Product date extracted from DATASTRIP:", product_date.date(), "\n")

# =====================================================================
# Roman conversion 
# =====================================================================

def int_to_roman(n):
    vals = [
        (1000, 'm'), (900, 'cm'), (500, 'd'), (400, 'cd'),
        (100, 'c'), (90, 'xc'), (50, 'l'), (40, 'xl'),
        (10, 'x'), (9, 'ix'), (5, 'v'), (4, 'iv'), (1, 'i')
    ]
    res = ""
    for v, s in vals:
        while n >= v:
            res += s
            n -= v
    return res


# =====================================================================
# Bulletin A
# =====================================================================


roman_year = int_to_roman(year - 1987)
doy = product_date.timetuple().tm_yday
index = (doy - 1) // 7 + 1

print(f"Bulletin A Roman year: {roman_year}")
print(f"Initial weekly index: {index}\n")

found = False

while index > 0:
    index_str = f"{index:03d}"
    url = f"https://datacenter.iers.org/data/6/bulletina-{roman_year}-{index_str}.txt"
    print("Trying bulletin:", url)

    response = requests.get(url)

    if response.status_code == 200:
        print("Bulletin found:", index_str)
        dest_file = os.path.join(DATA_DIR, f"bulletina-{roman_year}-{index_str}.txt")
        found = True
        break

    index -= 1

if not found:
    raise RuntimeError("No Bulletin A available for current or previous weeks.")

with open(dest_file, "wb") as f:
    f.write(response.content)

print("Downloaded:", dest_file)


Bulletin output directory: /home/avinas/Documents/sen2vm-fix/DATA
Removed old bulletin A: bulletina-xxxix-001.txt
Cleanup of old bulletins complete.

Product date extracted from DATASTRIP: 2026-01-04 

Bulletin A Roman year: xxxix
Initial weekly index: 1

Trying bulletin: https://datacenter.iers.org/data/6/bulletina-xxxix-001.txt
Bulletin found: 001
Downloaded: /home/avinas/Documents/sen2vm-fix/DATA/bulletina-xxxix-001.txt


# Sen2VM configuration

In [4]:
# === GENERATE CONFIG.JSON  ===

import os
import json
import re
import shutil
from numpy import double


USERCONF_DIR = os.path.join(WORKDIR, "UserConf")
os.makedirs(USERCONF_DIR, exist_ok=True)

print("UserConf directory:", USERCONF_DIR)

GEOID_DIR = os.path.join(WORKDIR, "DATA", "GEOID")
os.makedirs(GEOID_DIR, exist_ok=True)

print("Geoid directory:", GEOID_DIR)

# =====================================================
# 1. Extract mission from DATASTRIP
# =====================================================

datastrip_dir = os.path.join(PATH_L1B_DATA, "DATASTRIP")

if not os.path.isdir(datastrip_dir):
    raise RuntimeError(f"DATASTRIP directory not found: {datastrip_dir}")

datastrip_entries = os.listdir(datastrip_dir)
if not datastrip_entries:
    raise RuntimeError(f"No DATASTRIP found in: {datastrip_dir}")

datastrip_name = datastrip_entries[0]

match = re.match(r"(S2[A-C])_OPER_", datastrip_name)
if not match:
    raise RuntimeError("Cannot extract mission (S2A/S2B/S2C) from DATASTRIP name.")

mission = match.group(1)


# =====================================================
# 2. Docker paths inside /workspace
# =====================================================

safe_name = os.path.basename(PATH_L1B_DATA)

docker_l1b  = f"/workspace/DATA/{safe_name}"
docker_dem  = "/workspace/DATA/DEM"
docker_gipp = f"/workspace/DATA/sen2vm-gipp-database/{mission}"

# =====================================================
# 3. Geoid management
# =====================================================

# Notebook location (NOT relative to CWD)
notebook_dir = os.getcwd()

# Relative path to DEM_GEOID from notebook
internal_geoid_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "..", "..", "src", "test", "resources", "DEM_GEOID"
))

# If WORKDIR/DATA/GEOID is empty -> copy internal files
if len(os.listdir(GEOID_DIR)) == 0:
    print("GEOID directory is empty -> copying default geoid files...")
    for f in os.listdir(internal_geoid_dir):
        src = os.path.join(internal_geoid_dir, f)
        dst = os.path.join(GEOID_DIR, f)
        shutil.copy(src, dst)

# Detect .gtx inside GEOID_DIR
geoid_files = [f for f in os.listdir(GEOID_DIR) if f.lower().endswith(".gtx")]
if len(geoid_files) == 0:
    raise RuntimeError("No .gtx geoid file found in WORKDIR/DATA/GEOID.")

docker_geoid = f"/workspace/DATA/GEOID/{geoid_files[0]}"

# =====================================================
# 4. Locate IERS bulletin on host
# =====================================================

DATA_DIR = os.path.join(WORKDIR, "DATA")
iers_host = None

for f in os.listdir(DATA_DIR):
    if f.startswith("bulletin"):
        iers_host = os.path.join(DATA_DIR, f)
        break

if iers_host is None:
    raise RuntimeError("IERS bulletin not found inside WORKDIR/DATA directory.")

docker_iers = "/workspace/DATA/" + os.path.basename(iers_host)

# =====================================================
# 5. Build config dictionary
# =====================================================

config = {
    "l1b_product": docker_l1b,
    "gipp_folder": docker_gipp,
    "auto_gipp_selection": True,
    "grids_overwriting": True,
    "dem": docker_dem,
    "geoid": docker_geoid,
    "iers": docker_iers,
    "operation": GRID_MODE,
    "deactivate_available_refining": False,
    "steps": {
        "10m_bands": STEPS["10m_bands"],
        "20m_bands": STEPS["20m_bands"],
        "60m_bands": STEPS["60m_bands"]
    },
    "export_alt": True
}

# Add inverse block only if needed
if GRID_MODE == "inverse":
    config["inverse_location_additional_info"] = {
        "ul_x": double(LOCATION["ul_x"]),
        "ul_y": double(LOCATION["ul_y"]),
        "lr_x": double(LOCATION["lr_x"]),
        "lr_y": double(LOCATION["lr_y"]),
        "referential": f"EPSG:{UTM_EPSG}",
        "output_folder": "/workspace/DATA/Output"
    }

# =====================================================
# Save config.json
# =====================================================

config_path = os.path.join(USERCONF_DIR, "config.json")

with open(config_path, "w") as f:
    json.dump(config, f, indent=4)

print("Configuration file generated:")
print(config_path)


UserConf directory: /home/avinas/Documents/sen2vm-fix/UserConf
Geoid directory: /home/avinas/Documents/sen2vm-fix/DATA/GEOID
Configuration file generated:
/home/avinas/Documents/sen2vm-fix/UserConf/config.json


In [5]:
# === GENERATE PARAMS.JSON ===

import os
import json
import re

USERCONF_DIR = os.path.join(WORKDIR, "UserConf")
os.makedirs(USERCONF_DIR, exist_ok=True)

print("UserConf directory:", USERCONF_DIR)

# =====================================================
# Locate GRANULE folders
# =====================================================
GR_TARGET_DIR = os.path.join(PATH_L1B_DATA, "GRANULE")

if not os.path.exists(GR_TARGET_DIR):
    raise RuntimeError("GRANULE directory not found inside L1B SAFE.")

granule_folders = [
    os.path.join(GR_TARGET_DIR, d)
    for d in os.listdir(GR_TARGET_DIR)
    if os.path.isdir(os.path.join(GR_TARGET_DIR, d))
]

print("Found", len(granule_folders), "granule folders.")

# =====================================================
# Extract detectors and bands from JP2
# =====================================================
detectors = set()
bands = set()

pattern = r"_D(\d+)_B(\d{1,2}[A]?)\.jp2$"

for granule in granule_folders:
    img_data_dir = os.path.join(granule, "IMG_DATA")

    if not os.path.isdir(img_data_dir):
        continue

    for fname in os.listdir(img_data_dir):
        match = re.search(pattern, fname)
        if match:
            detectors.add(match.group(1))
            bands.add(f"B{match.group(2)}")

detectors = sorted(detectors)
bands = sorted(bands)

print("Detected detectors:", detectors)
print("Detected bands:", bands)

# =====================================================
# Write params.json
# =====================================================
params = {
    "detectors": detectors,
    "bands": bands
}

params_path = os.path.join(USERCONF_DIR, "params.json")

with open(params_path, "w") as f:
    json.dump(params, f, indent=4)

print("params.json written to:", params_path)


UserConf directory: /home/avinas/Documents/sen2vm-fix/UserConf
Found 119 granule folders.
Detected detectors: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
Detected bands: ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B8A']
params.json written to: /home/avinas/Documents/sen2vm-fix/UserConf/params.json


# Sen2VM run

In [6]:
# === RUN SEN2VM (Docker: BUILD + RUN + CLEAN) ===

import os
import subprocess


dockerfile_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "..", ".."
))

config_inside = "/workspace/UserConf/config.json"
params_inside = "/workspace/UserConf/params.json"

# =====================================================
# 1. BUILD DOCKER IMAGE
# =====================================================

print(f"Building Docker image 'sen2vm' from: {dockerfile_dir}")

cmd_build = [
    "docker", "build",
    "-t", "sen2vm",
    dockerfile_dir
]

print("Command:", " ".join(cmd_build), "\n")
subprocess.run(cmd_build, check=True)
print("Docker image built successfully.\n")

# =====================================================
# 2. RUN SEN2VM CONTAINER
# =====================================================

cmd_run = [
    "docker", "run",
    "--rm",
    "-v", f"{WORKDIR}:/workspace",  
    "sen2vm",
    "-c", config_inside,
    "-p", params_inside
]

print("Running Docker container...\n")
print("Command:", " ".join(cmd_run), "\n")

subprocess.run(cmd_run, check=True)

print("\nDocker execution complete.\n")

# =====================================================
# 3. REMOVE DOCKER IMAGE
# =====================================================

print("Removing Docker image 'sen2vm'...")

subprocess.run(["docker", "rmi", "-f", "sen2vm"], check=True)

print("Docker image removed.\n")

Building Docker image 'sen2vm' from: /home/avinas/Documents/sen2vm-fix/sen2vm-core
Command: docker build -t sen2vm /home/avinas/Documents/sen2vm-fix/sen2vm-core 



#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 324B done
#1 DONE 0.0s

#2 [internal] load metadata for ghcr.io/sen2vm/sen2vm-build-env:latest
#2 DONE 1.5s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/3] FROM ghcr.io/sen2vm/sen2vm-build-env:latest@sha256:a5369801b50064bf3c5c88dd9698f8762facc30f4d6b226aa1c6350dc1156200
#4 resolve ghcr.io/sen2vm/sen2vm-build-env:latest@sha256:a5369801b50064bf3c5c88dd9698f8762facc30f4d6b226aa1c6350dc1156200 0.0s done
#4 DONE 0.1s

#4 [1/3] FROM ghcr.io/sen2vm/sen2vm-build-env:latest@sha256:a5369801b50064bf3c5c88dd9698f8762facc30f4d6b226aa1c6350dc1156200
#4 sha256:90d6acbb058f4d86e99e863e354c4a94a04d6c9bf800e0738994990ef106a4ed 0B / 264.19MB 0.2s
#4 sha256:90d6acbb058f4d86e99e863e354c4a94a04d6c9bf800e0738994990ef106a4ed 16.78MB / 264.19MB 1.2s
#4 sha256:90d6acbb058f4d86e99e863e354c4a94a04d6c9bf800e0738994990ef106a4ed 33.5

Docker image built successfully.

Running Docker container...

Command: docker run --rm -v /home/avinas/Documents/sen2vm-fix:/workspace sen2vm -c /workspace/UserConf/config.json -p /workspace/UserConf/params.json 



2026-01-13 14:04:27 [INFO   ] Start Sen2VM 
2026-01-13 14:04:27 [INFO   ] Parsing file /workspace/UserConf/config.json 
2026-01-13 14:04:27 [INFO   ] Reading IERS file at: /workspace/DATA/bulletina-xxxix-001.txt 
2026-01-13 14:04:27 [INFO   ] Parsing file /workspace/UserConf/params.json 
2026-01-13 14:04:27 [INFO   ] Detectors list: [DETECTOR_1, DETECTOR_2, DETECTOR_3, DETECTOR_4, DETECTOR_5, DETECTOR_6, DETECTOR_7, DETECTOR_8, DETECTOR_9, DETECTOR_10, DETECTOR_11, DETECTOR_12] 
2026-01-13 14:04:27 [INFO   ] Bands list: [BAND_1, BAND_2, BAND_3, BAND_4, BAND_5, BAND_6, BAND_7, BAND_8, BAND_9, BAND_10, BAND_11, BAND_12, BAND_8A] 
2026-01-13 14:04:27 [INFO   ] Find the following datastrip metadata file: /workspace/DATA/S2A_MSIL1B_20260104T125121_N0511_R095_20260108T155119.SAFE/DATASTRIP/S2A_OPER_MSI_L1B_DS_2APS_20260104T175623_S20260104T125115_N05.11/S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml 
2026-01-13 14:04:28 [INFO   ] Initializing: copy of the Orekit-data: /Sen2vm/


Docker execution complete.

Removing Docker image 'sen2vm'...
Untagged: sen2vm:latest
Deleted: sha256:9d4589b853b5515c58e279df732d49ce5db5429daa429744a769f958a0de9af4
Docker image removed.



# Generate Orthorectification images

In [ ]:
import os
import subprocess
import glob

# =====================================================
# Locate product name
# =====================================================
product = os.path.basename(os.path.normpath(PATH_L1B_DATA))

# =====================================================
# Locate XML
# =====================================================
xml_list = glob.glob(
    os.path.join(
        PATH_L1B_DATA,
        "DATASTRIP",
        "S2*",
        "S2*_MTD_L1B_DS_*.xml"
    )
)

if len(xml_list) == 0:
    raise RuntimeError("No DATASTRIP MTD XML found")

xml_path = xml_list[0]
xml_name = os.path.basename(xml_path)

# XML path inside docker (relative, after cd)
xml_docker = f"./{xml_name}"

print("Using XML:", xml_path)

# =====================================================
# Locate VRTs
# =====================================================
vrt_list = glob.glob(
    os.path.join(PATH_L1B_DATA, "DATASTRIP", "S2*", "GEO_DATA", "*.vrt")
)

if not vrt_list:
    raise RuntimeError("No VRT files found in GEO_DATA")

vrt_names = [os.path.splitext(os.path.basename(v))[0] for v in vrt_list]

print("Found VRTs:", vrt_names)

# =====================================================
# Output directories
# =====================================================
OUTDIR = os.path.join(WORKDIR, "DATA", "GDAL_OUTPUT_ORTHO")
os.makedirs(OUTDIR, exist_ok=True)

OUTDIR_DOCKER = "/workspace/DATA/GDAL_OUTPUT_ORTHO"

# =====================================================
# Build GDAL docker
# =====================================================
notebook_dir = os.path.dirname(os.getcwd())
dockerfile_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "src",
    "gdal-latest"
))

print("\n=== BUILDING GDAL LATEST CONTAINER ===\n")
cmd_build = [
    "docker", "build",
    "--platform=linux/amd64",
    "-t", "gdal-latest",
    dockerfile_dir
]

print("Command:", " ".join(cmd_build), "\n")
subprocess.run(cmd_build, check=True)
print("GDAL image built successfully.\n")

# =====================================================
# Generate gdal_ortho.sh
# =====================================================
gdal_script_path = os.path.join(WORKDIR, "src", "gdal_ortho.sh")
os.makedirs(os.path.dirname(gdal_script_path), exist_ok=True)

vrt_array = " ".join([f'"{v}"' for v in vrt_names])

# Get STEPS values for resolution
step_10m = STEPS["10m_bands"]
step_20m = STEPS["20m_bands"]
step_60m = STEPS["60m_bands"]

with open(gdal_script_path, "w") as f:
    f.write(f"""#!/bin/bash
set +e

cd /workspace/DATA/{product}

OUT_ORTHO="/workspace/DATA/GDAL_OUTPUT_ORTHO"
OUT_MOSAIC="/workspace/DATA/GDAL_OUTPUT_MOSAIC"

mkdir -p "$OUT_ORTHO"
mkdir -p "$OUT_MOSAIC"

XML="{xml_docker}"

# =====================================================
# Function to get band resolution (Issue #61)
# =====================================================
get_band_resolution() {{
    local vrt_name="$1"
    # Extract band from VRT name (e.g., ..._D09_B01 -> B01)
    local band=$(echo "$vrt_name" | grep -oE '_B[0-9]{{1,2}}[A]?' | sed 's/_//')
    
    case "$band" in
        B02|B03|B04|B08)
            echo {step_10m}  # 10m bands
            ;;
        B05|B06|B07|B8A|B11|B12)
            echo {step_20m}  # 20m bands
            ;;
        B01|B09|B10)
            echo {step_60m}  # 60m bands
            ;;
        *)
            echo {step_10m}  # Default to 10m
            ;;
    esac
}}

echo "=== ORTHORECTIFICATION ==="

for VRT in {vrt_array}; do
    BASENAME="$VRT"
    OUT="$OUT_ORTHO/${{BASENAME}}_ortho.tif"

    echo "----------------------------------------"
    echo "Processing VRT: $VRT"
    echo "----------------------------------------"

    rm -f "$OUT"

    # Get resolution for this band (Issue #61)
    RESOLUTION=$(get_band_resolution "$VRT")
    echo "Band resolution: $RESOLUTION m"

    # Orthorectification with explicit resolution and nodata
    gdalwarp \\
        SENTINEL2_L1B_WITH_GEOLOC:./$XML:$VRT \\
        "$OUT" \\
        -t_srs EPSG:{UTM_EPSG} \\
        -te {LOCATION["ul_x"]} {LOCATION["lr_y"]} {LOCATION["lr_x"]} {LOCATION["ul_y"]} \\
        -r cubic \\
        -tr $RESOLUTION -$RESOLUTION \\
        -dstnodata 0 \\
        -co COMPRESS=LZW \\
        -co TILED=YES \\
        -overwrite

    echo ""
done

echo ""
echo "=== MOSAIC GENERATION ==="
echo ""

for BAND in {" ".join(ORTHO_SETTINGS["keep_bands"])}; do
    echo "----------------------------------------"
    echo "Creating mosaic for band: $BAND"
    echo "----------------------------------------"

    INPUT_FILES=($(ls $OUT_ORTHO/*_${{BAND}}_ortho.tif 2>/dev/null))

    if [ ${{#INPUT_FILES[@]}} -eq 0 ]; then
        echo "No ortho images found for band $BAND"
        continue
    fi

    OUTPUT="$OUT_MOSAIC/ORTHO_mosaic_${{BAND}}.tif"

    # Use gdal_merge.py instead of gdalwarp to avoid double resampling 
    # All ortho images should have the same resolution and geometry
    # No resampling needed, just assembly
    gdal_merge.py \\
        -o "$OUTPUT" \\
        -of GTiff \\
        -co COMPRESS=LZW \\
        -co TILED=YES \\
        -ot UInt16 \\
        -n 0 \\
        -a_nodata 0 \\
        "${{INPUT_FILES[@]}}"

    echo " Mosaic written -> $OUTPUT"
    echo ""
done

echo "=== GDAL processing complete ==="
""")

os.chmod(gdal_script_path, 0o755)
print("Generated:", gdal_script_path)

# =====================================================
# Run GDAL docker
# =====================================================
print("\n=== RUNNING GDAL PROCESSING ===\n")

cmd_run = [
    "docker", "run",
    "--rm",
    "-v", f"{WORKDIR}:/workspace",
    "gdal-latest",
    "/workspace/src/gdal_ortho.sh"
]

print("Command:", " ".join(cmd_run), "\n")
subprocess.run(cmd_run, check=True)
print("\nGDAL ortho + mosaic complete.\n")

# =====================================================
# Cleanup docker image
# =====================================================
print("Removing gdal-latest image...\n")
subprocess.run(["docker", "rmi", "-f", "gdal-latest"], check=True)
print("GDAL image removed.\n")


Using XML: /home/avinas/Documents/sen2vm-fix/DATA/S2A_MSIL1B_20260104T125121_N0511_R095_20260108T155119.SAFE/DATASTRIP/S2A_OPER_MSI_L1B_DS_2APS_20260104T175623_S20260104T125115_N05.11/S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml
Found VRTs: ['S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D04_B08', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B04', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D06_B11', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D01_B08', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B06', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D11_B12', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B10', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D03_B07', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D07_B10', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D03_B04', 'S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.28kB done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/ubuntu:22.04
#2 DONE 1.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/8] FROM docker.io/library/ubuntu:22.04@sha256:104ae83764a5119017b8e8d6218fa0832b09df65aae7d5a6de29a85d813da2fb
#4 resolve docker.io/library/ubuntu:22.04@sha256:104ae83764a5119017b8e8d6218fa0832b09df65aae7d5a6de29a85d813da2fb 0.0s done
#4 CACHED

#5 [2/8] RUN apt-get update && apt-get install -y     build-essential     git     cmake     pkg-config     curl     wget     zip     libcurl4-openssl-dev     libproj-dev     proj-bin     libgeos-dev     libsqlite3-dev     libtiff-dev     libjpeg-dev     libpng-dev     libzstd-dev     liblzma-dev     libopenjp2-7-dev     libboost-dev     python3     python3-pip     python3-dev     python3-numpy     swig     ninja-build 

GDAL image built successfully.

Generated: /home/avinas/Documents/sen2vm-fix/src/gdal_ortho.sh

=== RUNNING GDAL PROCESSING ===

Command: docker run --rm -v /home/avinas/Documents/sen2vm-fix:/workspace gdal-latest /workspace/src/gdal_ortho.sh 

=== ORTHORECTIFICATION ===
----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D04_B08
----------------------------------------
Band resolution: 10 m

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B04
----------------------------------------
Band resolution: 10 m


ERROR 1: Tile ././GRANULE/S2A_OPER_MSI_L1B_GR_2APS_20260104T175623_S20260104T125117_D04_N05.11/IMG_DATA/S2A_OPER_MSI_L1B_GR_2APS_20260104T175623_S20260104T125117_D04_B08.jp2 not found.
ERROR 4: Failed to open source file SENTINEL2_L1B_WITH_GEOLOC:././S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D04_B08



Creating output file that is 4844P x 2836L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B04 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D06_B11
----------------------------------------
Band resolution: 20 m
Creating output file that is 2422P x 1418L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D06_B11 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D01_B08
----------------------------------------
Band resolution: 10 m
Creating output file that is 4844P x 2836L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_O

...

Creating output file that is 2422P x 1418L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D08_B12 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D02_B11
----------------------------------------
Band resolution: 20 m
Creating output file that is 2422P x 1418L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D02_B11 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D02_B8A
----------------------------------------
Band resolution: 20 m
Creating output file that is 2422P x 1418L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_O

Warning 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,2422,1418, skipping.
Warning 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,2422,1418, skipping.


...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D07_B04
----------------------------------------
Band resolution: 10 m
Creating output file that is 4844P x 2836L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D07_B04 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D03_B08
----------------------------------------
Band resolution: 10 m
Creating output file that is 4844P x 2836L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D03_B08 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_

Warning 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,4844,2836, skipping.
Warning 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,807,473, skipping.
Warning 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 0,0,2422,1418, skipping.


...10...20...30...40...50...60...70...80...90...100 - done.

----------------------------------------
Processing VRT: S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D07_B03
----------------------------------------
Band resolution: 10 m
Creating output file that is 4844P x 2836L.
Processing S2A_OPER_MTD_L1B_DS_2APS_20260104T175623_S20260104T125115.xml:S2A_OPER_GEO_L1B_DS_2APS_20260104T175623_S20260104T125115_D07_B03 [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


=== MOSAIC GENERATION ===

----------------------------------------
Creating mosaic for band: B01
----------------------------------------
0...10...20...30...40...50...60...70...80...90...100 - done.
 Mosaic written -> /workspace/DATA/GDAL_OUTPUT_MOSAIC/ORTHO_mosaic_B01.tif

----------------------------------------
Creating mosaic for band: B02
----------------------------------------
0...10...20...30...40...50...60...70...80...90...100 - done.
 Mosaic written -> /workspace/DATA/GDAL_OUTPUT_MOS